In [ ]:
!pip install langchain
# from langchain_community.llms import HuggingFaceHub, HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import os
import torch

HF_API_KEY = 'INSERT_API_HERE'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HF_API_KEY'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
model_id = 'BioMistral/BioMistral-7B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id) #AutoModelForSeq2SeqLM if model is encoder-decoder

pipe = pipeline(
    "text-generation", #text2text-generation if model is encoder-decoder
    model = model,
    tokenizer = tokenizer,
    max_length = 100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
multi_template = """Answer the following questions one at a time.

Questions: {questions}
Answers:
"""

prompt = PromptTemplate(
    template=multi_template,
    input_variables=['questions']
)

llm_chain = LLMChain(
    llm = local_llm,
    prompt = prompt
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [32]:
# questions: This works but let's see if I can make it so that it handles .txt
# qs_str = [
#    {'questions': "Which NFL team won the Super Bowl in the 2010 season?"},
#    {'questions': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
#    {'questions': "Who was the 12th person on the moon?"},
#    {'questions': "How many eyes does a blade of grass have?"}
# ]
#

with open('elenchus.txt') as f:
    qs_str = f.readlines()

'{"questions":\t"my pain is so bad at times i am literally bald up in a ball and can not breathe and feels like someone is taking sharp knifes and stabbing me in my right mid adomin and stomach"},\n'

In [33]:
# answer:
res = llm_chain.generate(qs_str)

with open('output.txt') as fp:
    print(res)

TypeError: string indices must be integers